In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

Here is a simple recipe for making soju at home:

Ingredients:
- 1 cup of rice
- 1 cup of nuruk (Korean fermentation starter)
- 8 cups of water
- 1 tablespoon of yeast
- 1 tablespoon of sugar (optional)

Instructions:
1. Rinse the rice thoroughly until the water runs clear.
2. Soak the rice in water for about 1 hour, then drain.
3. Steam the rice until it becomes soft and fully cooked.
4. Let the cooked rice cool down to room temperature.
5. In a large container, combine the cooled rice, nuruk, and water.
6. Mix well until the nuruk is fully dissolved.
7. Cover the container with a clean cloth or plastic wrap and let it sit at room temperature for about 2-3 days.
8. After 2-3 days, you will notice bubbles forming on the surface, indicating fermentation.
9. Strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.
10. Dissolve yeast and sugar (optional) in a small amount of warm water and add it to the strained liquid.
11. Transfer the liquid to airtight bottl